In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
all = os.listdir("/kaggle/input/cyberbullying-dataset")
all

['twitter_sexism_parsed_dataset.csv',
 'youtube_parsed_dataset.csv',
 'kaggle_parsed_dataset.csv',
 'aggression_parsed_dataset.csv',
 'toxicity_parsed_dataset.csv',
 'attack_parsed_dataset.csv',
 'twitter_parsed_dataset.csv',
 'twitter_racism_parsed_dataset.csv']

In [3]:
df3 = pd.read_csv('/kaggle/input/cyberbullying-dataset/twitter_parsed_dataset.csv')
df3

,index,id,Text,Annotation,oh_label
0,5.74948705591165E+017,5.74948705591165E+017,@halalflaws @biebervalue @greenlinerzjm I read...,none,0.0
1,5.71917888690393E+017,5.71917888690393E+017,@ShreyaBafna3 Now you idiots claim that people...,none,0.0
2,3.90255841338601E+017,3.90255841338601E+017,"RT @Mooseoftorment Call me sexist, but when I ...",sexism,1.0
3,5.68208850655916E+017,5.68208850655916E+017,"@g0ssipsquirrelx Wrong, ISIS follows the examp...",racism,1.0
4,5.75596338802373E+017,5.75596338802373E+017,#mkr No No No No No No,none,0.0
...,...,...,...,...,...
16846,5.75606766236475E+017,5.75606766236475E+017,"Feeling so sorry for the girls, they should be...",none,0.0
16847,5.72333822886326E+017,5.72333822886326E+017,#MKR 'pretty good dishes we're happy with' - O...,none,0.0
16848,5.72326950057845E+017,5.72326950057845E+017,RT @colonelkickhead: Deconstructed lemon tart!...,none,0.0
16849,5.74799612642357E+017,5.74799612642357E+017,@versacezaynx @nyazpolitics @greenlinerzjm You...,none,0.0


In [7]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Sample DataFrame


# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the 'Category' column
df3['Encoded_Category'] = label_encoder.fit_transform(df3['Annotation'])
encoding_map = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

print("Encoded values and their corresponding categories:")
print(encoding_map)
# df3

Encoded values and their corresponding categories:
{'none': 0, 'racism': 1, 'sexism': 2, nan: 3}


In [8]:
df_final = df3.drop(['index','id','oh_label'],axis =1)

In [9]:
df_final

,Text,Annotation,Encoded_Category
0,@halalflaws @biebervalue @greenlinerzjm I read...,none,0
1,@ShreyaBafna3 Now you idiots claim that people...,none,0
2,"RT @Mooseoftorment Call me sexist, but when I ...",sexism,2
3,"@g0ssipsquirrelx Wrong, ISIS follows the examp...",racism,1
4,#mkr No No No No No No,none,0
...,...,...,...
16846,"Feeling so sorry for the girls, they should be...",none,0
16847,#MKR 'pretty good dishes we're happy with' - O...,none,0
16848,RT @colonelkickhead: Deconstructed lemon tart!...,none,0
16849,@versacezaynx @nyazpolitics @greenlinerzjm You...,none,0


In [4]:
df3.dropna(inplace=True)

In [5]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Sample DataFrame


# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the 'Category' column
df3['Encoded_Category'] = label_encoder.fit_transform(df3['Annotation'])
encoding_map = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

print("Encoded values and their corresponding categories:")
print(encoding_map)
# df3

Encoded values and their corresponding categories:
{'none': 0, 'racism': 1, 'sexism': 2}


In [6]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split

# Sample text data and labels
texts = df3['Text']  # List of texts
labels = df3['Encoded_Category']
texts

0        @halalflaws @biebervalue @greenlinerzjm I read...
1        @ShreyaBafna3 Now you idiots claim that people...
2        RT @Mooseoftorment Call me sexist, but when I ...
3        @g0ssipsquirrelx Wrong, ISIS follows the examp...
4                                   #mkr No No No No No No
                               ...                        
16846    Feeling so sorry for the girls, they should be...
16847    #MKR 'pretty good dishes we're happy with' - O...
16848    RT @colonelkickhead: Deconstructed lemon tart!...
16849    @versacezaynx @nyazpolitics @greenlinerzjm You...
16850    And before you protest that you're *not* mad, ...
Name: Text, Length: 16848, dtype: object

In [9]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split

# Sample text data and labels
texts = list(df3['Text'])  # List of texts
labels = list(df3['Encoded_Category'])  # List of corresponding labels (0 or 1 for binary classification)

# Split data into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)

# Tokenize input texts
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=128)

# Create PyTorch datasets
train_dataset = TensorDataset(
    torch.tensor(train_encodings['input_ids']),
    torch.tensor(train_encodings['attention_mask']),
    torch.tensor(train_labels)
)
val_dataset = TensorDataset(
    torch.tensor(val_encodings['input_ids']),
    torch.tensor(val_encodings['attention_mask']),
    torch.tensor(val_labels)
)

# Define DataLoader
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Load pre-trained BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

# Define optimizer and loss function
optimizer = AdamW(model.parameters(), lr=5e-5)
loss_fn = torch.nn.CrossEntropyLoss()

# Train the model
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

for epoch in range(5):  # Train for 3 epochs
    model.train()
    i = 0
    for batch in train_loader:
        input_ids, attention_mask, labels = tuple(t.to(device) for t in batch)
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
#         i+=1
#         if(i%100 == 0):
#             print(i/100)

    # Validation
    model.eval()
    val_accuracy = 0
    with torch.no_grad():
        for batch in val_loader:
            input_ids, attention_mask, labels = tuple(t.to(device) for t in batch)
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1)
            val_accuracy += torch.sum(preds == labels).item()
    val_accuracy /= len(val_dataset)
    print(f"Epoch {epoch + 1}: Validation Accuracy = {val_accuracy:.2f}")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1: Validation Accuracy = 0.85
Epoch 2: Validation Accuracy = 0.87
Epoch 3: Validation Accuracy = 0.86
Epoch 4: Validation Accuracy = 0.86
Epoch 5: Validation Accuracy = 0.85
